In [ ]:
from ollama import Client
from pydantic import BaseModel

class ExampleOutputClass(BaseModel):
    response : str
    evidences : list[str]


def multiply(a : int, b : int):
    """
    Multiples two numbers together.

    Args:
        a: First number.
        b: Second number.
    
    Returns:
        The result of multiplying a and b.
    """

    return a * b

client = Client()

available_function = {"multiply" : multiply}

response = client.chat(
    model="llama3.2:latest", 
    messages=[{"role": "user", "content": "What is five times 3? Your response MUST be in JSON format."}],
    tools = [multiply],
    format = ExampleOutputClass.model_json_schema(),
    options = {"temperature" : "0"}
)

print(response["message"]["content"])

pythonic_output = ExampleOutputClass.model_validate_json(response["message"]["content"])
print(pythonic_output)

for tool in response.message.tool_calls or []:
    function_to_call = available_function.get(tool.function.name)
    if function_to_call:
        print(f"Result: {function_to_call(**tool.function.arguments)}")

# Huggingface Function Calling

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# from huggingface_hub import login
# login(token = "")

def divide(a : float, b : float):
    """
    Gets result of dividing two numbers.

    Args:
        a: First number.
        b: Second number.
    """
    return a // b

tools = [divide]

model_name = "NousResearch/Hermes-2-Pro-Llama-3-8B"  # Replace with the appropriate Llama model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

prompt = """What is five divided by four?"""

content = [{"roler" : "user", "content" : prompt}]


tool_prompts = tokenizer.apply_chat_template(content, tools=tools, return_tensors="pt", return_dict=True, add_generation_prompt=True).to(model.device)
outputs = model.generate(**tool_prompts, max_length=250)

# Decode the generated text
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)